In [1]:
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.style
matplotlib.style.use('paperstyle.mpl')
import matplotlib.pyplot as plt

In [2]:
import bisect
import scipy as sp
import scipy.stats

In [3]:
def weighted_median(quantity, weights=None, alpha=0.5, axis=-1):
    if weights is None:
        weights = np.full_like(quantity, 1./np.shape(quantity)[axis])
    total = np.sum(weights, axis=axis)
    order = np.argsort(quantity, axis=axis)
    sorted_q = quantity[order]
    sorted_w = weights[order]
    cumulative_w = np.cumsum(sorted_w, axis=axis) / total

    i = bisect.bisect_left(cumulative_w, alpha) - 1 
    if i < 0 or i >= len(quantity):
        return None
    return (sorted_q[i]*sorted_w[i]*(1.0 - alpha) + sorted_q[i+1]*sorted_w[i+1]*(alpha))/(sorted_w[i]*(1.0 - alpha) + sorted_w[i+1]*(alpha))

In [4]:
# set the colormap and centre the colorbar
class MidpointNormalize(matplotlib.colors.Normalize):
    """
    Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        matplotlib.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

In [5]:
energy = (lambda x: (x[:-1]+x[1:])/2.)(np.logspace(0,11,89)) # in GeV
distance = 1./np.logspace(np.log10(1./1000), np.log10(1./(2e5)), 100) # in m
energy_bins = np.logspace(0,11,89) # in GeV
distance_diff = (np.log10(1./1000) - np.log10(1./(2e5))) / (100-1)
distance_bins = 1./np.logspace(np.log10(1./1000)+distance_diff/2., np.log10(1./(2e5))-distance_diff/2., 100+1) # in m

In [6]:
f = open('output.txt', 'r')
entries = np.zeros((len(energy), len(distance), 0)).tolist()
initial_energies = []
distances = []
final_energies = []
for line in f:
    items = line.split(' ')
    energy_i = int(items[0])
    distance_i = int(items[1])
    final_energy = float(items[2])/1e3
    if distance_i == len(distance):
        continue
    if energy_i == len(energy):
        continue
    entries[energy_i][distance_i].append(final_energy)
    initial_energies.append(energy[energy_i])
    distances.append(distance[distance_i])
    final_energies.append(final_energy)
initial_energies = np.array(initial_energies)
distances = np.array(distances)
final_energies = np.array(final_energies)
entries = np.array(entries)

In [7]:
def prep_2d_hist(x, y, x_edges, y_edges, quantity, f=None):
    x_mapping = np.digitize(x, bins=x_edges) - 1
    y_mapping = np.digitize(y, bins=y_edges) - 1
    n_x_edges = len(x_edges)
    n_y_edges = len(y_edges)
    n_x_bins = n_x_edges-1
    n_y_bins = n_y_edges-1
    print(n_x_bins, n_y_bins)
    #bin_masks = np.empty((n_y_bins, n_x_bins, len(x)))
    binned_quantity = np.empty((n_y_bins, n_x_bins, 0)).tolist()
    binned_result = np.empty((n_y_bins, n_x_bins)).tolist()
    for j in range(n_y_bins):
        for k in range(n_x_bins):
            mask = np.logical_and(x_mapping == k, y_mapping == j)
            #bin_masks[j,k,:] = mask
            binned_quantity[j][k] = quantity[mask]
            if f is not None:
                binned_result[j, k] = f(binned_quantity[j][k])
    X = np.array([x_edges]*(n_y_edges))
    Y = np.array([y_edges]*(n_x_edges)).T
    if f is not None:
        return X, Y, binned_quantity, binned_result
    else:
        return X, Y, binned_quantity

In [8]:
X, Y, binned_q = prep_2d_hist(initial_energies, distances, energy_bins, distance_bins, final_energies)

88 100


In [9]:
vmin = np.amin(final_energies[final_energies>0])
vmax = np.amax(final_energies)

In [10]:
median = np.array([[weighted_median(x, alpha=0.5) for x in l] for l in binned_q])
median[median == 0] = np.nan
cm = plt.get_cmap('plasma')
cm.set_bad('white')
fig, ax = plt.subplots(figsize=(7,5))
mesh = ax.pcolormesh(X,Y,median, cmap=cm, norm=matplotlib.colors.LogNorm(vmin=vmin, vmax=vmax))
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Initial Muon Energy [GeV]')
ax.set_ylabel(r'Muon Overburden [m]')
cb = fig.colorbar(mesh, ax=ax)
cb.ax.set_ylabel('Median Final Energy')
cb.ax.minorticks_off()
plt.tight_layout()
fig.savefig('./preach_median.pdf')
fig.clf()
plt.close(fig)

/home/austin/.local/lib/python3.6/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


In [70]:
median = np.array([[np.average(x) for x in l] for l in binned_q])
median[median == 0] = np.nan
cm = plt.get_cmap('plasma')
cm.set_bad('white')
fig, ax = plt.subplots(figsize=(7,5))
mesh = ax.pcolormesh(X,Y,median, cmap=cm, norm=matplotlib.colors.LogNorm(vmin=vmin, vmax=vmax))
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Initial Muon Energy [GeV]')
ax.set_ylabel(r'Muon Overburden [m]')
cb = fig.colorbar(mesh, ax=ax)
cb.ax.set_ylabel('Mean Final Energy')
cb.ax.minorticks_off()
plt.tight_layout()
fig.savefig('./preach_mean.pdf')
fig.clf()
plt.close(fig)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """
/home/austin/.local/lib/python3.6/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


In [71]:
q = np.array([[np.sqrt(np.var(x)) for x in l] for l in binned_q])
q[q == 0] = np.nan
cm = plt.get_cmap('plasma')
cm.set_bad('white')
fig, ax = plt.subplots(figsize=(7,5))
mesh = ax.pcolormesh(X,Y,q, cmap=cm, norm=matplotlib.colors.LogNorm(vmin=vmin, vmax=vmax))
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Initial Muon Energy [GeV]')
ax.set_ylabel(r'Muon Overburden [m]')
cb = fig.colorbar(mesh, ax=ax)
cb.ax.set_ylabel('StdDev Final Energy')
cb.ax.minorticks_off()
plt.tight_layout()
fig.savefig('./preach_stddev.pdf')
fig.clf()
plt.close(fig)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """
/home/austin/.local/lib/python3.6/site-packages/matplotlib/colors.py:1110: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


In [72]:
q = np.array([[sp.stats.skew(x) for x in l] for l in binned_q])
print(q)
q[q == 0] = np.nan
cm = plt.get_cmap('RdBu')
cm.set_bad('white')
fig, ax = plt.subplots(figsize=(7,5))
mesh = ax.pcolormesh(X,Y,q, cmap=cm, norm=MidpointNormalize(midpoint=0))
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Initial Muon Energy [GeV]')
ax.set_ylabel(r'Muon Overburden [m]')
cb = fig.colorbar(mesh, ax=ax)
cb.ax.set_ylabel('Skew')
cb.ax.minorticks_off()
plt.tight_layout()
fig.savefig('./preach_skew.pdf')
fig.clf()
plt.close(fig)

[[ 0.          0.          0.         ... -0.79358434 -0.7968242
  -0.77240475]
 [ 0.          0.          0.         ... -0.74352868 -0.76034103
  -0.72583897]
 [ 0.          0.          0.         ... -0.67339341 -0.68523038
  -0.67161724]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


In [74]:
f = open('1km_output.txt', 'r')
entries = np.zeros((len(energy), len(distance), 0)).tolist()
initial_energies = []
distances = []
final_energies = []
for line in f:
    items = line.split(' ')
    energy_i = int(items[0])
    distance_i = int(items[1])
    final_energy = float(items[2])/1e3
    if distance_i == len(distance):
        continue
    if energy_i == len(energy):
        continue
    entries[energy_i][distance_i].append(final_energy)
    initial_energies.append(energy[energy_i])
    distances.append(distance[distance_i])
    final_energies.append(final_energy)
initial_energies = np.array(initial_energies)
distances = np.array(distances)
final_energies = np.array(final_energies)
entries = np.array(entries)

In [75]:
final_energies

array([0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.43951e+10,
       7.11019e+10, 7.24547e+10])

In [76]:
initial_energies

array([1.55590042e+00, 1.55590042e+00, 1.55590042e+00, ...,
       8.74947105e+10, 8.74947105e+10, 8.74947105e+10])

In [77]:
def mode(inputData, axis=None, dtype=None):
    """
    Robust estimator of the mode of a data set using the half-sample mode.
    .. versionadded: 1.0.3
    """

    if axis is not None:
        fnc = lambda x: mode(x, dtype=dtype)
        dataMode = np.apply_along_axis(fnc, axis, inputData)
    else:
        # Create the function that we can use for the half-sample mode
        def _hsm(data):
            if data.size == 1:
                return data[0]
            elif data.size == 2:
                return data.mean()
            elif data.size == 3:
                i1 = data[1] - data[0]
                i2 = data[2] - data[1]
                if i1 < i2:
                    return data[:2].mean()
                elif i2 > i1:
                    return data[1:].mean()
                else:
                    return data[1]
            else:
                wMin = data[-1] - data[0]
                N = int(data.size / 2 + data.size % 2)
                j = None
                for i in range(0, N):
                    w = data[i+N-1] - data[i]
                    if w < wMin:
                        wMin = w
                        j = i
                if j is None:
                    return data.mean()
                return _hsm(data[j:j+N])

        data = inputData.ravel()
        if type(data).__name__ == "MaskedArray":
            data = data.compressed()
        if dtype is not None:
            data = data.astype(dtype)

        # The data need to be sorted for this to work
        data = np.sort(data)

        # Find the mode
        dataMode = _hsm(data)

    return dataMode

def calc_min_interval(x, alpha):
    """Internal method to determine the minimum interval of
    a given width
    Assumes that x is sorted numpy array.
    """
    n = len(x)
    cred_mass = 1.0 - alpha

    interval_idx_inc = int(np.floor(cred_mass * n))
    n_intervals = n - interval_idx_inc
    interval_width = x[interval_idx_inc:] - x[:n_intervals]

    if len(interval_width) == 0:
        raise ValueError('Too few elements for interval calculation')

    min_idx = np.argmin(interval_width)
    hdi_min = x[min_idx]
    hdi_max = x[min_idx + interval_idx_inc]
    return hdi_min, hdi_max

def hpd(x, alpha=0.05, transform=lambda x: x):
    """Calculate highest posterior density (HPD) of array for given alpha. The HPD is the
    minimum width Bayesian credible interval (BCI).
    :Arguments:
      x : Numpy array
          An array containing MCMC samples
      alpha : float
          Desired probability of type I error (defaults to 0.05)
      transform : callable
          Function to transform data (defaults to identity)
    """
    # Make a copy of trace
    x = transform(x.copy())

    # For multivariate node
    if x.ndim > 1:

        # Transpose first, then sort
        tx = np.transpose(x, list(range(x.ndim))[1:] + [0])
        dims = np.shape(tx)

        # Container list for intervals
        intervals = np.resize(0.0, dims[:-1] + (2,))

        for index in make_indices(dims[:-1]):

            try:
                index = tuple(index)
            except TypeError:
                pass

            # Sort trace
            sx = np.sort(tx[index])

            # Append to list
            intervals[index] = calc_min_interval(sx, alpha)

        # Transpose back before returning
        return np.array(intervals)

    else:
        # Sort univariate node
        sx = np.sort(x)

        return np.array(calc_min_interval(sx, alpha))

def interval(arr, proportion=scipy.special.erf(1.0/np.sqrt(2.0)), min=None, max=None):
    """
    Compute distribution mode and the HPD that contains `proportion` of the mass.
    """
    x = hpd(arr, alpha=(1.0-proportion))
    if x[0] <= np.amin(arr):
        if min is not None:
            x[0] = min
    if x[-1] <= np.amax(arr):
        if max is not None:
            x[1] = max
    return x[0], mode(arr), x[1]

In [85]:
indices = np.digitize(initial_energies, bins=energy_bins) - 1
masks = [indices == i for i in range(len(energy_bins) - 1)]
energy_centers = (lambda x: (x[:-1]+x[1:])/2.)(energy_bins)
interval_final_energies = [interval(initial_energies[m]-final_energies[m], 0.9) if np.sum(m)>0 else (energy_centers[i],)*3 for i,m in enumerate(masks)]
y = [v[1] for v in interval_final_energies]
print(y)
yerr = np.array([(v[1]-v[0], v[2]-v[1]) for v in interval_final_energies]).T
#err_final_energies = [np.sqrt(np.sum(final_energies[m]**2)/float(np.sum(m))) if np.sum(m)>0 else 0.0 for m in masks]
print(avg_final_energies)
fig, ax = plt.subplots(figsize=(7,5))
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Initial Muon Energy [GeV]')
ax.set_ylabel(r'$-\Delta E$ [GeV] after 1km')
#ax.hist(energy_centers, bins=energy_bins, weights=y, histtype='step', color='black')
ax.errorbar(energy_centers, y, yerr=yerr, capsize=2, color='blue')
fig.tight_layout()
fig.savefig('muon_energy.pdf')

[1.166760716081662, 1.5559004211011238, 2.07482655785029, 2.7668256829150173, 3.6896213472271024, 4.920189143094658, 6.561177672614022, 8.749471046662277, 11.66760716081662, 15.55900421101123, 20.748265578502888, 27.668256829150167, 36.896213472271015, 49.201891430946574, 65.61177672614028, 87.49471046662279, 116.67607160816623, 155.5900421101123, 207.48265578502878, 276.6825682915018, 277.3521347227101, 301.47391430946567, 318.2957672614024, 357.4906046662279, 392.03471608166194, 429.6104211011234, 530.7215578502888, 590.1206829150174, 725.8313472271011, 890.4391430946562, 1112.772672614025, 1305.2210466622782, 2054.8171608166203, 2324.5042110112336, 3061.71557850289, 4175.556829150173, 5786.413472271004, 6876.041430946563, 8540.626726140246, 13062.010466622785, 15927.071608166196, 21416.04211011232, 29388.15578502891, 45009.568291501724, 51897.13472271012, 70600.41430946568, 82834.26726140245, 142244.60466622794, 161910.7160816621, 236545.42110112356, 311796.55785028916, 404815.68291

/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # Remove the CWD from sys.path while we load stuff.


In [63]:
initial_energies[-10:]

array([8.74947105e+10, 8.74947105e+10, 8.74947105e+10, 8.74947105e+10,
       8.74947105e+10, 8.74947105e+10, 8.74947105e+10, 8.74947105e+10,
       8.74947105e+10, 8.74947105e+10])

In [64]:
final_energies[-10:]

array([6.23467e+10, 6.77479e+10, 6.07689e+10, 6.12505e+10, 7.35273e+10,
       7.24957e+10, 6.45639e+10, 1.43951e+10, 7.11019e+10, 7.24547e+10])